<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'


from DatasetCIFAR.data_set import Dataset

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


# Import packages

In [0]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Subset, DataLoader

# Hyper-Parameters

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
BATCH_SIZE = 256
NUM_WORKERS = 100
TASK_SIZE = 10
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LR = 1
NUM_EPOCHS = 30

# Define Network

In [0]:
resNet = models.resnet18(pretrained=False)
resNet = resNet.to(DEVICE)

# Define Data Loader

In [6]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train=True),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

test_loader = torch.utils.data.DataLoader(
    Dataset(train=False),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

Extracting data/cifar-100-python.tar.gz to data


NameError: ignored

# Define training parameters

In [0]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(resNet.parameters(), lr=LR)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# Train phase

In [7]:
test_indexes = []
for task in range(0, 100, TASK_SIZE):
  print("Task classes {} to {}".format(task, task + TASK_SIZE))

  train_indexes = train_loader.dataset.__getIndexesGroups__(task)
  test_indexes = test_indexes.append(test_loader.dataset.__getIndexesGroups__(task))

  train_dataset = Subset(Dataset(train=True), train_indexes)
  test_dataset = Subset(Dataset(train=False), test_indexes)
  resNet.fc = nn.Linear(512, task)

  for epoch in range(NUM_EPOCHS):
    for images, labels in train_loader:
    # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      print(labels)
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = resNet(images)

      loss = criterion(outputs, labels)

      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients
      print("At step ", str(task), " and at epoch = ", epoch, " the loss is = ", loss)
  scheduler.step() 

Task classes 0 to 10


NameError: ignored